In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.1.0,org.apache.spark:spark-sql-kafka-0-10_2.11:2.1.0,com.databricks:spark-avro_2.11:3.2.0 pyspark-shell'

In [1]:
from pytwitter import StreamApi
import json
import os
import credentials # Import api/access_token keys from credentials.py
import settings # Import related setting constants from settings.py 
import pykafka

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext


In [16]:
class MyStream(StreamApi):
    def __init__(self, bearer_token):
        StreamApi.__init__(self, bearer_token=bearer_token)
        self.__file_index = 0
    
    def on_tweet(self, tweet):
        print(tweet)
        tweets_file_path = 'tweets_json/{}.json'.format(self.__file_index)
        with open(tweets_file_path, 'w', encoding='utf-8') as outfile:
                json.dump(tweet, outfile, ensure_ascii=False)
        while os.path.exists(tweets_file_path) and os.stat(tweets_file_path).st_size > 2**10:
            self.__file_index += 1 
            tweets_file_path = 'tweets_json/{}.json'.format(self.__file_index)

            

In [13]:
rules = {
        "add": [
            {"value": "cat has:media", "tag": "cats with media"},
            {"value": "cat has:media -grumpy", "tag": "happy cats with media"}
        ]
     }

In [17]:
stream = MyStream(bearer_token=credentials.BEARER_TOKEN)
stream.manage_rules(rules=rules)
stream.search_stream(return_json=True)
# stream.sample_stream(return_json=True)

{'data': {'id': '1545043971441041409', 'text': 'RT @cat_mparadise: https://t.co/zfEm8AMnqm'}, 'matching_rules': [{'id': '1545037781386153986', 'tag': 'happy cats with media'}, {'id': '1545037781386153987', 'tag': 'cats with media'}]}
{'data': {'id': '1545043973240016896', 'text': 'RT @Dexerto: There is a game coming out where you play as a cat roaming a dystopian future city\n\nhttps://t.co/jdnG3IirRJ'}, 'matching_rules': [{'id': '1545037781386153986', 'tag': 'happy cats with media'}, {'id': '1545037781386153987', 'tag': 'cats with media'}]}
{'data': {'id': '1545043976964558849', 'text': 'RT @twtCats: if i were a cat https://t.co/lhrGQVvx91'}, 'matching_rules': [{'id': '1545037781386153986', 'tag': 'happy cats with media'}, {'id': '1545037781386153987', 'tag': 'cats with media'}]}
{'data': {'id': '1545043975815409664', 'text': 'RT @Dexerto: There is a game coming out where you play as a cat roaming a dystopian future city\n\nhttps://t.co/jdnG3IirRJ'}, 'matching_rules': [{'id': '1545037

KeyboardInterrupt: 